In [1]:
import pandas as pd
import numpy as np
import datetime
import re
from collections import Counter

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

# Eda and null-analysis to craft final dataframe

In [3]:
totaldf = pd.read_csv('data/totaldf.csv')

In [4]:
totaldf.head(1)

,title,link,release_date,been_sale,orig_price,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
0,NieR Replicant™ ver.1.22474487139...,https://store.steampowered.com/app/1113560/Nie...,"Apr 23, 2021",0,$59.99,NaN,NaN,9.0,"['Great Soundtrack', 'Action', 'RPG', 'Adventu...",20.0,"['Action', 'Adventure', 'RPG', 'Square Enix']",4.0,Square Enix


In [5]:
totaldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15109 entries, 0 to 15108
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          15109 non-null  object 
 1   link           15109 non-null  object 
 2   release_date   15068 non-null  object 
 3   been_sale      15109 non-null  int64  
 4   orig_price     15109 non-null  object 
 5   num_reviews    5793 non-null   object 
 6   review_rating  14152 non-null  object 
 7   num_languages  6928 non-null   float64
 8   tags           15109 non-null  object 
 9   num_tags       15109 non-null  float64
 10  genres         15109 non-null  object 
 11  num_genres     15109 non-null  float64
 12  publisher      15083 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 1.5+ MB


In [6]:
# lets look at only the games that had total reviews listed

reviewsdf = totaldf[totaldf.num_reviews.isna()==False]

In [7]:
reviewsdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5793 entries, 1 to 15108
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          5793 non-null   object 
 1   link           5793 non-null   object 
 2   release_date   5792 non-null   object 
 3   been_sale      5793 non-null   int64  
 4   orig_price     5793 non-null   object 
 5   num_reviews    5793 non-null   object 
 6   review_rating  5793 non-null   object 
 7   num_languages  2193 non-null   float64
 8   tags           5793 non-null   object 
 9   num_tags       5793 non-null   float64
 10  genres         5793 non-null   object 
 11  num_genres     5793 non-null   float64
 12  publisher      5792 non-null   object 
dtypes: float64(3), int64(1), object(9)
memory usage: 633.6+ KB


# Number of Reviews

removing the comma and setting the type as int

In [8]:
reviewsdf.num_reviews = reviewsdf.num_reviews.str.replace(',','').astype(int)

In [9]:
reviewsdf.head(2)

,title,link,release_date,been_sale,orig_price,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
1,OUTRIDERS,https://store.steampowered.com/app/680420/OUTR...,"Apr 1, 2021",0,$59.99,36837,Mixed,13.0,"['RPG', 'Action', 'Adventure', 'Co-op', 'Third...",20.0,"['Action', 'Adventure', 'RPG']",3.0,Square Enix
2,It Takes Two,https://store.steampowered.com/app/1426210/It_...,"Mar 26, 2021",0,$39.99,15688,Overwhelmingly Positive,12.0,"['Co-op', 'Adventure', 'Puzzle', '3D Platforme...",20.0,"['Action', 'Adventure', 'Hazelight']",3.0,Electronic Arts


In [10]:
reviewsdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5793 entries, 1 to 15108
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          5793 non-null   object 
 1   link           5793 non-null   object 
 2   release_date   5792 non-null   object 
 3   been_sale      5793 non-null   int64  
 4   orig_price     5793 non-null   object 
 5   num_reviews    5793 non-null   int64  
 6   review_rating  5793 non-null   object 
 7   num_languages  2193 non-null   float64
 8   tags           5793 non-null   object 
 9   num_tags       5793 non-null   float64
 10  genres         5793 non-null   object 
 11  num_genres     5793 non-null   float64
 12  publisher      5792 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 633.6+ KB


# Year released
One missing value
creating year released column for modeling

In [11]:
# only one title from reviews df is missing its release date
# has a high number of reviews, gettig release date online.
reviewsdf[reviewsdf.release_date.isna()]

,title,link,release_date,been_sale,orig_price,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
784,Don't Starve,https://store.steampowered.com/app/219740/Dont...,NaN,0,$9.99,87348,Overwhelmingly Positive,NaN,"['Survival', 'Open World Survival Craft', 'Cra...",20.0,"['Adventure', 'Indie', 'Simulation', 'Klei Ent...",4.0,Klei Entertainment


In [12]:
reviewsdf.loc[reviewsdf.title == "Don't Starve",['release_date']] = 'Apr 23, 2013'

In [13]:
reviewsdf.loc[reviewsdf.title == "Don't Starve"]

,title,link,release_date,been_sale,orig_price,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
784,Don't Starve,https://store.steampowered.com/app/219740/Dont...,"Apr 23, 2013",0,$9.99,87348,Overwhelmingly Positive,NaN,"['Survival', 'Open World Survival Craft', 'Cra...",20.0,"['Adventure', 'Indie', 'Simulation', 'Klei Ent...",4.0,Klei Entertainment


In [14]:
reviewsdf.release_date = pd.to_datetime(reviewsdf.release_date)

In [15]:
reviewsdf['year_released'] = reviewsdf.release_date.dt.year

In [16]:
reviewsdf.head(2)

,title,link,release_date,been_sale,orig_price,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher,year_released
1,OUTRIDERS,https://store.steampowered.com/app/680420/OUTR...,2021-04-01,0,$59.99,36837,Mixed,13.0,"['RPG', 'Action', 'Adventure', 'Co-op', 'Third...",20.0,"['Action', 'Adventure', 'RPG']",3.0,Square Enix,2021
2,It Takes Two,https://store.steampowered.com/app/1426210/It_...,2021-03-26,0,$39.99,15688,Overwhelmingly Positive,12.0,"['Co-op', 'Adventure', 'Puzzle', '3D Platforme...",20.0,"['Action', 'Adventure', 'Hazelight']",3.0,Electronic Arts,2021


# Price

removing the dollar sign and setting type as float

In [17]:
reviewsdf.orig_price = reviewsdf.orig_price.str.replace('$','')

In [18]:
# there was one game that was 'Free', replaced with 0 to make the column a float
reviewsdf.loc[reviewsdf.orig_price == 'Free',['orig_price']] = 0

In [19]:
reviewsdf.orig_price = reviewsdf.orig_price.astype(float)


In [20]:
reviewsdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5793 entries, 1 to 15108
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          5793 non-null   object        
 1   link           5793 non-null   object        
 2   release_date   5793 non-null   datetime64[ns]
 3   been_sale      5793 non-null   int64         
 4   orig_price     5793 non-null   float64       
 5   num_reviews    5793 non-null   int64         
 6   review_rating  5793 non-null   object        
 7   num_languages  2193 non-null   float64       
 8   tags           5793 non-null   object        
 9   num_tags       5793 non-null   float64       
 10  genres         5793 non-null   object        
 11  num_genres     5793 non-null   float64       
 12  publisher      5792 non-null   object        
 13  year_released  5793 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(3), object(6)
memory usage: 678

# Languages

There are a large number of games without number of languages offered in. Going to create a separate column with the nulls filled in as average value. Can try modeling with and without this feature.

In [21]:
reviewsdf.num_languages.mean()

11.225262197902417

In [22]:
reviewsdf['languages_filled'] = reviewsdf.num_languages.fillna(reviewsdf.num_languages.mean())

In [23]:
reviewsdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5793 entries, 1 to 15108
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             5793 non-null   object        
 1   link              5793 non-null   object        
 2   release_date      5793 non-null   datetime64[ns]
 3   been_sale         5793 non-null   int64         
 4   orig_price        5793 non-null   float64       
 5   num_reviews       5793 non-null   int64         
 6   review_rating     5793 non-null   object        
 7   num_languages     2193 non-null   float64       
 8   tags              5793 non-null   object        
 9   num_tags          5793 non-null   float64       
 10  genres            5793 non-null   object        
 11  num_genres        5793 non-null   float64       
 12  publisher         5792 non-null   object        
 13  year_released     5793 non-null   int64         
 14  languages_filled  5793 

# Publisher

extracting top 15 publishers, grouping the rest into 'other'

In [24]:
top_pub_list = list(reviewsdf.publisher.value_counts().head(15).keys())

In [25]:
top_pub_list

['SEGA',
 'Electronic Arts',
 'Ubisoft',
 'Square Enix',
 'BANDAI NAMCO Entertainment',
 'Disney',
 'Sokpop Collective',
 'THQ Nordic',
 'Devolver Digital',
 'Paradox Interactive',
 'tinyBuild',
 'Activision',
 'Winged Cloud',
 '2K',
 'Capcom']

In [26]:
def big_publisher(string):
    if string in (top_pub_list):
        return string
    else:
        return 'other'
    

In [27]:
reviewsdf['top_pubs'] = reviewsdf.publisher.apply(big_publisher)

In [28]:
# creating dummies from top pubs column

pub_dummies = pd.get_dummies(reviewsdf.top_pubs, drop_first=True)


In [29]:
# merging back into reviewsdf

reviewsdf = pd.merge(reviewsdf, pub_dummies, left_index=True, right_index=True)

# Review Rating

Creating numerical values for the ratings.

In [30]:
reviewsdf.review_rating.value_counts()

Very Positive              2703
Mostly Positive             941
Mixed                       711
Overwhelmingly Positive     444
Positive                    432
3 user reviews               67
2 user reviews               64
4 user reviews               58
Mostly Negative              57
1 user reviews               55
5 user reviews               55
6 user reviews               51
8 user reviews               42
7 user reviews               38
9 user reviews               37
No user reviews              21
Negative                     12
Very Negative                 4
Overwhelmingly Negative       1
Name: review_rating, dtype: int64

In [31]:
def rating_converter (rating):
    if rating == 'Overwhelmingly Positive':
        return 10
    elif rating =='Very Positive':
        return 9
    elif rating == 'Positive':
        return 8
    elif rating == 'Mostly Positive':
        return 7
    elif rating == 'Mixed':
        return 6
    elif rating == 'Mostly Negative':
        return 5
    elif rating == 'Negative':
        return 4
    elif rating == 'Very Negative':
        return 3
    elif rating == 'Overwhelmingly negative':
        return 2
    else:
        return 1



In [32]:
reviewsdf['rating_val'] = reviewsdf.review_rating.apply(rating_converter)

In [33]:
reviewsdf.head(2)

,title,link,release_date,been_sale,orig_price,num_reviews,review_rating,num_languages,tags,num_tags,...,Paradox Interactive,SEGA,Sokpop Collective,Square Enix,THQ Nordic,Ubisoft,Winged Cloud,other,tinyBuild,rating_val
1,OUTRIDERS,https://store.steampowered.com/app/680420/OUTR...,2021-04-01,0,59.99,36837,Mixed,13.0,"['RPG', 'Action', 'Adventure', 'Co-op', 'Third...",20.0,...,0,0,0,1,0,0,0,0,0,6
2,It Takes Two,https://store.steampowered.com/app/1426210/It_...,2021-03-26,0,39.99,15688,Overwhelmingly Positive,12.0,"['Co-op', 'Adventure', 'Puzzle', '3D Platforme...",20.0,...,0,0,0,0,0,0,0,0,0,10


# Tags

The tags are stored as a list for each row. Extracting them into one major list, then getting value counts, and manually creating dummy variables for the top 30 publishers

In [34]:
# there were extra characters when trying a normal split, using regex expression to only capture the actual 
#tags

regex = re.compile('[^1-9a-zA-Z]')
prelim_list = []

def megalist(list):
    
    for items in (list.split()):
        prelim_list.append(regex.sub('',items)) 
    


In [35]:
reviewsdf.tags.apply(megalist)

1        None
2        None
3        None
4        None
5        None
         ... 
15103    None
15104    None
15105    None
15107    None
15108    None
Name: tags, Length: 5793, dtype: object

In [36]:
Counter(prelim_list).most_common()[:30]

[('Singleplayer', 3608),
 ('Indie', 3308),
 ('Action', 3231),
 ('Adventure', 2886),
 ('Casual', 2066),
 ('Strategy', 2010),
 ('Multiplayer', 1996),
 ('RPG', 1887),
 ('Simulation', 1711),
 ('Atmospheric', 1605),
 ('Horror', 1536),
 ('2D', 1534),
 ('Soundtrack', 1392),
 ('Great', 1317),
 ('Story', 1276),
 ('Rich', 1276),
 ('World', 1193),
 ('Shooter', 1179),
 ('Puzzle', 1153),
 ('TurnBased', 1054),
 ('Open', 1050),
 ('FirstPerson', 1022),
 ('Survival', 1017),
 ('Fantasy', 936),
 ('Funny', 920),
 ('Coop', 895),
 ('Platformer', 893),
 ('Early', 868),
 ('Access', 868),
 ('Sim', 830)]

In [37]:
# takes the 30 most used tags
tags_count = Counter(prelim_list).most_common()[:30]

top_tags = []

for tags in tags_count:
    top_tags.append(tags[0])

In [38]:
#creating the columns

for tags in top_tags:
    reviewsdf[tags+'_tag'] = 0

In [39]:
# filling in the values for the new columns

for items in top_tags:
    reviewsdf.loc[reviewsdf.tags.str.contains(items),[items+'_tag']] = 1

In [40]:
reviewsdf.head(3)

,title,link,release_date,been_sale,orig_price,num_reviews,review_rating,num_languages,tags,num_tags,...,Open_tag,FirstPerson_tag,Survival_tag,Fantasy_tag,Funny_tag,Coop_tag,Platformer_tag,Early_tag,Access_tag,Sim_tag
1,OUTRIDERS,https://store.steampowered.com/app/680420/OUTR...,2021-04-01,0,59.99,36837,Mixed,13.0,"['RPG', 'Action', 'Adventure', 'Co-op', 'Third...",20.0,...,1,0,0,0,0,0,0,0,0,0
2,It Takes Two,https://store.steampowered.com/app/1426210/It_...,2021-03-26,0,39.99,15688,Overwhelmingly Positive,12.0,"['Co-op', 'Adventure', 'Puzzle', '3D Platforme...",20.0,...,0,0,0,0,0,0,1,0,0,0
3,Valheim,https://store.steampowered.com/app/892970/Valh...,2021-02-02,0,19.99,222855,Overwhelmingly Positive,12.0,"['Open World Survival Craft', 'Online Co-Op', ...",20.0,...,1,0,1,0,0,0,0,1,1,0


# Genre

Same as tags

In [41]:
prelim_list = []

reviewsdf.genres.apply(megalist)


1        None
2        None
3        None
4        None
5        None
         ... 
15103    None
15104    None
15105    None
15107    None
15108    None
Name: genres, Length: 5793, dtype: object

In [42]:
Counter(prelim_list).most_common()[:30]

[('Indie', 3233),
 ('Action', 2226),
 ('Adventure', 2131),
 ('Casual', 1524),
 ('Simulation', 1342),
 ('Strategy', 1233),
 ('RPG', 1131),
 ('', 567),
 ('Early', 535),
 ('Access', 535),
 ('Games', 503),
 ('Sports', 241),
 ('Racing', 208),
 ('Studios', 165),
 ('Entertainment', 129),
 ('Massively', 113),
 ('Multiplayer', 113),
 ('Studio', 113),
 ('Interactive', 113),
 ('Inc', 94),
 ('Software', 82),
 ('Game', 57),
 ('Utilities', 57),
 ('Feral', 57),
 ('Ltd', 53),
 ('Mac', 49),
 ('Design', 49),
 ('Illustration', 45),
 ('Sokpop', 43),
 ('Collective', 43)]

In [43]:
genres_count = Counter(prelim_list).most_common()[:30]

top_genres = []

for genres in genres_count:
    top_genres.append(genres[0])

In [44]:
# Creating the columns

for genres in top_genres:
    reviewsdf[genres+'_genre'] = 0

In [45]:
# Filling them in

for items in top_genres:
    reviewsdf.loc[reviewsdf.genres.str.contains(items),[items+'_genre']] = 1

In [46]:
reviewsdf.head(5)

,title,link,release_date,been_sale,orig_price,num_reviews,review_rating,num_languages,tags,num_tags,...,Software_genre,Game_genre,Utilities_genre,Feral_genre,Ltd_genre,Mac_genre,Design_genre,Illustration_genre,Sokpop_genre,Collective_genre
1,OUTRIDERS,https://store.steampowered.com/app/680420/OUTR...,2021-04-01,0,59.99,36837,Mixed,13.0,"['RPG', 'Action', 'Adventure', 'Co-op', 'Third...",20.0,...,0,0,0,0,0,0,0,0,0,0
2,It Takes Two,https://store.steampowered.com/app/1426210/It_...,2021-03-26,0,39.99,15688,Overwhelmingly Positive,12.0,"['Co-op', 'Adventure', 'Puzzle', '3D Platforme...",20.0,...,0,0,0,0,0,0,0,0,0,0
3,Valheim,https://store.steampowered.com/app/892970/Valh...,2021-02-02,0,19.99,222855,Overwhelmingly Positive,12.0,"['Open World Survival Craft', 'Online Co-Op', ...",20.0,...,0,0,0,0,0,0,0,0,0,0
4,7 Days to Die,https://store.steampowered.com/app/251570/7_Da...,2013-12-13,1,24.99,163711,Very Positive,14.0,"['Early Access', 'Survival', 'Voxel', 'Zombies...",20.0,...,0,0,0,0,0,0,0,0,0,0
5,Titanfall® 2,https://store.steampowered.com/app/1237970/Tit...,2020-06-18,1,29.99,57126,Overwhelmingly Positive,10.0,"['FPS', 'Multiplayer', 'Action', 'Shooter', 'M...",20.0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
reviewsdf.shape

(5793, 92)

In [48]:
reviewsdf.columns

Index(['title', 'link', 'release_date', 'been_sale', 'orig_price',
       'num_reviews', 'review_rating', 'num_languages', 'tags', 'num_tags',
       'genres', 'num_genres', 'publisher', 'year_released',
       'languages_filled', 'top_pubs', 'Activision',
       'BANDAI NAMCO Entertainment', 'Capcom', 'Devolver Digital', 'Disney',
       'Electronic Arts', 'Paradox Interactive', 'SEGA', 'Sokpop Collective',
       'Square Enix', 'THQ Nordic', 'Ubisoft', 'Winged Cloud', 'other',
       'tinyBuild', 'rating_val', 'Singleplayer_tag', 'Indie_tag',
       'Action_tag', 'Adventure_tag', 'Casual_tag', 'Strategy_tag',
       'Multiplayer_tag', 'RPG_tag', 'Simulation_tag', 'Atmospheric_tag',
       'Horror_tag', '2D_tag', 'Soundtrack_tag', 'Great_tag', 'Story_tag',
       'Rich_tag', 'World_tag', 'Shooter_tag', 'Puzzle_tag', 'TurnBased_tag',
       'Open_tag', 'FirstPerson_tag', 'Survival_tag', 'Fantasy_tag',
       'Funny_tag', 'Coop_tag', 'Platformer_tag', 'Early_tag', 'Access_tag',
       '

In [49]:
reviewsdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5793 entries, 1 to 15108
Data columns (total 92 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   title                       5793 non-null   object        
 1   link                        5793 non-null   object        
 2   release_date                5793 non-null   datetime64[ns]
 3   been_sale                   5793 non-null   int64         
 4   orig_price                  5793 non-null   float64       
 5   num_reviews                 5793 non-null   int64         
 6   review_rating               5793 non-null   object        
 7   num_languages               2193 non-null   float64       
 8   tags                        5793 non-null   object        
 9   num_tags                    5793 non-null   float64       
 10  genres                      5793 non-null   object        
 11  num_genres                  5793 non-null   float64    

In [50]:
# saving as reviewsdf.csv 

reviewsdf.to_csv(r'/Users/michaelharnett/Desktop\reviewsdf.csv', index=False)

IndentationError: unexpected indent (<ipython-input-50-6ade2dca7322>, line 3)

# Creating just numerical df for modeling

## Feature Editing
dropping unneccsary columns such as title and link for their game page, and setting target as first colunn

In [49]:
reviewsdf.columns

Index(['title', 'link', 'release_date', 'been_sale', 'orig_price',
       'num_reviews', 'review_rating', 'num_languages', 'tags', 'num_tags',
       'genres', 'num_genres', 'publisher', 'year_released',
       'languages_filled', 'top_pubs', 'Activision',
       'BANDAI NAMCO Entertainment', 'Capcom', 'Devolver Digital', 'Disney',
       'Electronic Arts', 'Paradox Interactive', 'SEGA', 'Sokpop Collective',
       'Square Enix', 'THQ Nordic', 'Ubisoft', 'Winged Cloud', 'other',
       'tinyBuild', 'rating_val', 'Singleplayer_tag', 'Indie_tag',
       'Action_tag', 'Adventure_tag', 'Casual_tag', 'Strategy_tag',
       'Multiplayer_tag', 'RPG_tag', 'Simulation_tag', 'Atmospheric_tag',
       'Horror_tag', '2D_tag', 'Soundtrack_tag', 'Great_tag', 'Story_tag',
       'Rich_tag', 'World_tag', 'Shooter_tag', 'Puzzle_tag', 'TurnBased_tag',
       'Open_tag', 'FirstPerson_tag', 'Survival_tag', 'Fantasy_tag',
       'Funny_tag', 'Coop_tag', 'Platformer_tag', 'Early_tag', 'Access_tag',
       '

In [50]:
modeldf = reviewsdf.drop(columns =[ 'title', 'link', 'release_date', 'review_rating',
                                  'num_languages', 'tags', 'genres','publisher',
                                  'top_pubs'])

In [51]:
modeldf

,been_sale,orig_price,num_reviews,num_tags,num_genres,year_released,languages_filled,Activision,BANDAI NAMCO Entertainment,Capcom,...,Software_genre,Game_genre,Utilities_genre,Feral_genre,Ltd_genre,Mac_genre,Design_genre,Illustration_genre,Sokpop_genre,Collective_genre
1,0,59.99,36837,20.0,3.0,2021,13.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,39.99,15688,20.0,3.0,2021,12.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,19.99,222855,20.0,6.0,2021,12.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,24.99,163711,20.0,7.0,2013,14.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,29.99,57126,20.0,2.0,2020,10.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15103,0,0.99,114,19.0,5.0,2015,11.225262,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15104,1,4.99,35,3.0,3.0,2016,7.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15105,1,4.99,50,2.0,2.0,2016,7.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15107,0,0.99,90,11.0,4.0,2015,11.225262,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
cols = list(modeldf.columns)

In [58]:
cols.insert(0, cols.pop(cols.index('num_reviews')))

In [60]:
modeldf = modeldf[cols]

In [61]:
modeldf.head()

,num_reviews,been_sale,orig_price,num_tags,num_genres,year_released,languages_filled,Activision,BANDAI NAMCO Entertainment,Capcom,...,Software_genre,Game_genre,Utilities_genre,Feral_genre,Ltd_genre,Mac_genre,Design_genre,Illustration_genre,Sokpop_genre,Collective_genre
1,36837,0,59.99,20.0,3.0,2021,13.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15688,0,39.99,20.0,3.0,2021,12.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,222855,0,19.99,20.0,6.0,2021,12.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,163711,1,24.99,20.0,7.0,2013,14.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,57126,1,29.99,20.0,2.0,2020,10.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### saving model df to use in modeling notebook

In [63]:
modeldf.to_csv(r'/Users/michaelharnett/Desktop\modeldf.csv', index=False)